In [53]:
import numpy as np
import pandas as pd
import json
from shapely.geometry import shape, Point



,INCIDENT_TYPE_DESCRIPTION,FROMDATE,WEAPONTYPE,Shooting,DOMESTIC,Year,Month,DAY_WEEK,Location
0,RESIDENTIAL BURGLARY,07/08/2012 06:00:00 AM,Other,No,No,2012,7,Sunday,"(42.34638135, -71.10379454)"
1,AGGRAVATED ASSAULT,07/08/2012 06:03:00 AM,Firearm,Yes,No,2012,7,Sunday,"(42.31684135, -71.07458456)"
2,ROBBERY,07/08/2012 06:26:00 AM,Firearm,No,No,2012,7,Sunday,"(42.34284135, -71.09698955)"
3,COMMERCIAL BURGLARY,07/08/2012 06:56:00 AM,Other,No,No,2012,7,Sunday,"(42.3164411, -71.06582908)"
4,ROBBERY,07/08/2012 07:15:00 AM,Firearm,No,No,2012,7,Sunday,"(42.27051636, -71.11989955)"
5,ROBBERY,07/08/2012 07:32:00 AM,Firearm,Yes,No,2012,7,Sunday,"(42.31328183, -71.0530059)"
6,ROBBERY,07/08/2012 07:50:00 AM,Firearm,No,No,2012,7,Sunday,"(42.32425136, -71.08620956)"
7,SIMPLE ASSAULT,07/08/2012 07:50:00 AM,Unarmed,No,No,2012,7,Sunday,"(42.34924634, -71.06378456)"
8,MedAssist,07/08/2012 07:53:00 AM,Unarmed,No,No,2012,7,Sunday,"(42.35174635, -71.16590953)"
9,MedAssist,07/08/2012 08:05:00 AM,Unarmed,No,No,2012,7,Sunday,"(42.25938275, -71.11729354)"


In [169]:
# somerville happiness data
happy = pd.read_csv('somerville_happy.csv')
happy.columns = [
            'ID', 'Year', 'Current Happy',
           'Overall Life Satisfaction',
           'Somerville Satisfaction',
           'Individual Similarity To Acquaitances 2011',
           'Rely on advice or self for decision making 2011',
           'Neighborhood satisfaction',
           'Proud to be Somerville resident 2015',
           'City Services Availability Rating 2015',
           'Availability of affordable housing 2011',
           'Cost of Housing Rating',
           'Rate cost of public schools 2011',
           'Rate overall cost of public schools 2011',
           'Rate beauty or physical setting 2011',
           'Rate beauty or physical setting 2013',
           'Rate effectiveness of local police 2011_2013',
           'Trust in local police 2015',
           'Rate maintenance of streets, sidewalks and squares 2013',
           'Rate maintenance of streets, sidewalks and squares 2015',
           'Availability of social communities and events',
           'Safety walking in neighborhood at night 2013',
           'Safety walking in community at night 2015',
           'Rate beauty or physical setting of neighborhood 2015',
           'Satisfaction with appearance of parks and squares 2013',
           'Satisfaction with local parks and squares',
           'Gender', 'Gender 2011', 'Age',
           'marital status 2011',
           'Household languages (non-english) 2015',
           'Ethnicity 2011_2013',
           'Is Hispanic 2013',
           'Ethnicity 2015',
           'Lives with Minors',
           'Housing Status',
           'Plans to leave Somerville within 2 years',
           'Years Lived in Somerville',
           'Annual Household Income',
           'Neighborhood', 'Is Student', 'Ward',
           'Precinct'
    ]

def normalize_income(income):
    if income == 'Less than $10,000':
        return income
    
    if income in ['$25,000 to $49,999', '40,000 - $49,999',
                  '$10,000 to $24,999', '30,000 - $39,999',
                  '20,000 - $29,999', '10,000 - $19,999'
                  ]:
        return '<50k'
    elif income in ['50,000 - $59,999', '60,000 - $69,999',
                   '70,000 - $79,999', '80,000 - $89,999',
                   '90,000 - $99,999', '$50,000 to $74,999',
                   '$75,000 to $99,999']:
        return '50k-100k'
    else:
        return '>100k'
    
def norm_eth(eth):
    ethnicity = eth.split(',')[0].split('/')[0].strip()
    if 'white' in ethnicity or 'asian' in ethnicity:
        if 'black' in ethnicity or 'African-American' in ethnicity:
            return 'black'
        return 'white/asian'
    elif 'Black' in ethnicity or 'African-American' in ethnicity:
        return 'Black'
    elif 'R' == ethnicity:
        return None
    elif 'American Indian' == ethnicity:
        return 'Native American'
    else:
        return ethnicity

    

happy['Income Bracket'] = happy['Annual Household Income'].apply(normalize_income)
happy['ETHNICITY'] = (happy[['Ethnicity 2011_2013', 'Ethnicity 2015']]
                      .fillna('')
                      .sum(axis=1)
                      .apply(norm_eth))



def get_income_data():
    incomes = pd.DataFrame(happy[happy['Year'] == 2011]['Income Bracket'].value_counts())
    incomes['2015'] = happy[happy['Year'] == 2015]['Income Bracket'].value_counts().sort_index()
    incomes.columns = ['2011', '2015']
    incomes['2011 %'] = incomes['2011'].apply(lambda x: x/float(incomes['2011'].sum()))
    incomes['2015 %'] = incomes['2015'].apply(lambda x: x/float(incomes['2015'].sum()))
    return incomes

def get_happy_data(year, income_bracket):
    # happiness data
    # happy[(happy['Year'] == 2015) & (happy['Income Bracket'] == '<50k')]['Current Happy'].mean()
    current_happy = (happy
        .where((happy['Year'] == year) & (happy['Income Bracket'] == income_bracket))
        ['Overall Life Satisfaction']
        .value_counts()
        .sort_index()
        .reset_index())
    current_happy['index'] = pd.to_numeric(current_happy['index'], 
                                           errors='coerce')
    current_happy['Overall Life Satisfaction'] = pd.to_numeric(current_happy['Overall Life Satisfaction'], 
                                           errors='coerce')

    current_happy = (current_happy
     .dropna()
     .where(current_happy['index'] < 11)
     .dropna(how='any')
     .sort_index())

    current_happy['Less than 6'] = current_happy['index'].apply(lambda x: x < 6)
    current_happy.groupby('Less than 6').sum()
    
writer = pd.ExcelWriter('happy_cleaned.xls')
happy.to_excel(writer, 'Sheet1')
get_income_data().to_excel(writer, 'Sheet2')
writer.save()
    




In [172]:
def get_age_bracket(age):
    if age < 13 or age > 80:
        return None
    if age < 18:
        return '<18'
    if age < 30:
        return '18 - 30'
    
    return ' 30+'
        

fio = pd.read_csv('bpd_fio.csv')

# init columns
fio['FRISKED'] = fio['OUTCOME'].astype(str).apply(lambda x: x.find('F') != -1)
fio['SEIZED'] = fio['OUTCOME'].astype(str).apply(lambda x: x.find('S') != -1)
fio['AGE_BRACKET'] = fio['AGE_AT_FIO_CORRECTED'].apply(get_age_bracket)
fio['YEAR'] = fio['FIO_DATE'].apply(lambda x: x[6:10])
fio['MONTH'] = fio['FIO_DATE'].apply(lambda x: x[0:2])
fio['DAY'] = fio['FIO_DATE'].apply(lambda x: x[3:5])

#total stop breakdown
fio[fio['YEAR'] == '2015']['LOCATION'].value_counts()
# % black
# fio.RACE_DESC.value_counts() / len(fio.index)

# # % black by neighborhood
# (fio.groupby(['CITY', 'RACE_DESC']).size() / fio.groupby(['CITY']).size()).sort_values(ascending=False)

# # city-wide makeup w/ no priors
# no_priors = fio[fio['PRIORS'] == 'NO']
# no_priors.RACE_DESC.value_counts() / len(no_priors.index)
# # neighborhood dist with no priors
# (no_priors.groupby(['CITY', 'RACE_DESC']).size() / no_priors.groupby(['CITY']).size())
# # city dist w/ priors
# has_priors = fio[fio['PRIORS'] == 'YES']
# has_priors.RACE_DESC.value_counts() / len(has_priors.index)
# # stop reasons
# (fio.groupby(['RACE_DESC', 'BASIS']).size() / fio.groupby('RACE_DESC').size())
# # terrorism
# t = fio[fio.TERRORISM == 'YES']
# t.groupby(['RACE_DESC', 'STOP_REASONS']).size()
# # % w/ priors by race *** black people that were stopped more likely to have priors
# (fio.groupby(['RACE_DESC', 'PRIORS']).size() / fio.groupby('RACE_DESC').size())
# # age distribution - median age is 25
# fio.AGE_AT_FIO_CORRECTED.median()
# # median age by race - blacks have lowest median age (24) & whites have the highest (31)
# fio[(~fio['AGE_BRACKET'].isnull()) & (fio['YEAR'] == '2015')].groupby('RACE_DESC').agg('median')['AGE_AT_FIO_CORRECTED']

 BOYLSTON ST                     112
 WASHINGTON ST                    91
00 MASSACHUSETTS AV               88
618 SHAWMUT AV                    86
500 GENEVA AV                     54
 TREMONT ST                       54
00 WASHINGTON ST                  54
279 CENTRE ST                     49
00 TREMONT ST                     49
00 SHAWMUT AV                     44
9 DITMUS CT                       37
54 ANNUNCIATION RD                34
 COLUMBIA RD                      32
 BOWER ST                         29
18 LATTIMORE CT                   26
934 PARKER ST                     24
00 BOYLSTON ST                    22
 MT PLEASANT AV                   22
200 HANCOCK ST                    21
 BLUE HILL AV                     20
4 WAKULLAH ST                     20
70 ANNUNCIATION RD                19
 HARRISON AV                      19
 ESSEX ST                         18
60 ANNUNCIATION RD                18
00 ALBANY ST                      18
00 COLUMBUS AV                    17
0

In [193]:
# crime_data = pd.read_csv('crime_july2012-august2015.csv', dtype='unicode')

# with open('site/public/json/neighborhoods.json') as f:
#     js = json.load(f)

# polygons = [shape(feature['geometry']) for feature in js['features']]
# def get_neighborhood(x):
#     lat, lon, count = x[0], x[1], x[2]
#     if count % 500 == 0:
#         print(count)
#     point = Point(lon, lat)
#         # check each polygon to see if it contains the point
#     for i, polygon in enumerate(polygons):
#         if polygon.contains(point):
#             return js['features'][i]['properties']['Name']
        
#     return None
            
# crime_data['location_lat'] = crime_data['Location'].apply(lambda x: float(x.split(',')[0][1:]))
# crime_data['location_lon'] = crime_data['Location'].apply(lambda x: float(x.split(',')[-1][:-1]))
# crime_data['count'] = crime_data.reset_index().index
# crime_data['Neighborhood'] = crime_data[['location_lat', 'location_lon', 'count']].apply(get_neighborhood, axis=1)
# crime_data.columns[:]
shootings_2015 = crime_data[(crime_data['Year'] == '2015') & (crime_data['Shooting'] == 'Yes')]
shootings_2015 = pd.DataFrame(shootings_2015.Neighborhood.value_counts()).sort_index()
shootings_2015 = shootings_2015.reset_index()
shootings_2015.columns = ['Neighborhood', 'Shootings2015']
shootings_2015 = shootings_2015.set_index('Neighborhood')
shootings_2015

    

,Shootings2015
Neighborhood,
Back Bay,1
Dorchester,41
Downtown,1
East Boston,2
Hyde Park,5
Jamaica Plain,6
Longwood Medical Area,2
Mattapan,8
Mission Hill,3


In [183]:
# crime_data_2015.to_csv('crime_data_2015.csv', sep='@')
def is_violent(row):
    desc, weapon = row
    if 'Assault' in desc or 'ASSAULT' in desc:
        return True
    elif weapon not in ['None', 'Unarmed', 'Other']:
        return True
    return False
        
crime_data['violent'] = crime_data[['INCIDENT_TYPE_DESCRIPTION', 'WEAPONTYPE']].apply(is_violent, axis=1)
crime_data[(crime_data['Year'] == '2015') & (crime_data['Neighborhood'] == 'Back Bay') & (crime_data['violent'] == True)]

crime_data.to_csv('crime_data_cleaned.csv', sep='@')

In [191]:
crime_data_2015 = pd.read_csv('crime_data_cleaned.csv', sep='@').where(crime_data['Year'] == '2015')
violent_crimes = (pd.DataFrame(crime_data_2015[crime_data_2015['violent'] == True]
                              .Neighborhood
                              .value_counts())
                            .reset_index())


violent_crimes.columns = ['Neighborhood', 'Violent Crimes']
violent_crimes = violent_crimes.set_index('Neighborhood').sort_index()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,...,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10
0,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; Total: - White alone,Margin of Error; Total: - White alone,Estimate; Total: - Black or African American a...,Margin of Error; Total: - Black or African Ame...,Estimate; Total: - American Indian and Alaska ...,...,Estimate; Total: - Native Hawaiian and Other P...,Margin of Error; Total: - Native Hawaiian and ...,Estimate; Total: - Some other race alone,Margin of Error; Total: - Some other race alone,Estimate; Total: - Two or more races:,Margin of Error; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two ra...,Margin of Error; Total: - Two or more races: -...,Estimate; Total: - Two or more races: - Two ra...,Margin of Error; Total: - Two or more races: -...
1,1400000US25025000100,25025000100,"Census Tract 1, Suffolk County, Massachusetts",3671,364,2348,390,178,151,0,...,0,12,556,393,58,55,14,20,44,52
2,1400000US25025000201,25025000201,"Census Tract 2.01, Suffolk County, Massachusetts",3284,325,2729,300,143,117,0,...,0,12,106,88,106,77,36,44,70,59
3,1400000US25025000202,25025000202,"Census Tract 2.02, Suffolk County, Massachusetts",3587,493,2342,396,365,188,0,...,0,12,295,324,35,44,15,34,20,31
4,1400000US25025000301,25025000301,"Census Tract 3.01, Suffolk County, Massachusetts",2712,367,2146,320,50,36,0,...,0,12,154,116,0,12,0,12,0,12


In [195]:
census = pd.read_csv('ma_census_2015.csv', dtype='unicode', skiprows=1)
tract_data = pd.read_csv('tract_data.csv', delimiter=' ')
# pop = pd.read_csv('pop_total.csv', dtype='unicode', skiprows=1)
race = pd.read_csv('race_hispanic.csv', skiprows=1)


def get_tract_id(label):
    s = label.split(',')
    return s[0][12:]

def wavg(group, avg_name, weight_name):
    """ 
    http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

def pct(group, num):
    n = group[num]
    d = group['pop']
    return n.sum() / d.sum()
    
census['tract'] = census['Geography'].apply(get_tract_id)
census = (census
          .rename(columns = {'Id2':'GEOID'})
          .set_index('GEOID'))

# population data
# pop = (pop
#           .rename(columns = {'Id2':'GEOID'})
#           .set_index('GEOID'))
# pop = pd.DataFrame(pop['Estimate; Total'])

#race data
race = (race
          .rename(columns = {'Id2':'GEOID'})
          .set_index('GEOID'))

race.columns
race = race[[
        'Estimate; Total:',
       u'Estimate; Not Hispanic or Latino: - White alone',
       u'Estimate; Not Hispanic or Latino: - Black or African American alone',
       u'Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone',
       u'Estimate; Not Hispanic or Latino: - Asian alone',
       u'Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone',
       u'Estimate; Not Hispanic or Latino: - Some other race alone',
       u'Estimate; Not Hispanic or Latino: - Two or more races:',
         'Estimate; Hispanic or Latino:'
    ]]

race_columns = ['pop', 'White', 'Black', 'AmericanIndianNativeAlaskan', 'Asian', 'NativeHawaiianPacificIslander',
                'OtherRace', 'TwoOrMoreRaces', 'HispanicLatino']
race.columns = race_columns

race = race.set_index(race.index.astype(str))

# census tract -> neighhborhood map
tract_data['GEOID'] = tract_data['GEOID'].astype(str)
tract_data = tract_data.set_index('GEOID')

# join above
census = census.join(tract_data, how='inner')
census
census = census[['Neighborhood', 'tract',
                 'Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed', 
                 'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All families'
                ]]
census.columns = ['Neighborhood', 'tract', 'unemployment rate', 'percent families below poverty level']
census = census.join(race, how='inner').replace('-', np.nan)

census[['pop', 'unemployment rate', 'percent families below poverty level']] = census[['pop', 'unemployment rate', 'percent families below poverty level']].astype(float)

# weighted avg percents to get neighborhood percents
employment_by_neighborhood = census.groupby('Neighborhood').apply(wavg, 'unemployment rate', 'pop')
poverty_by_neighborhood = census.groupby('Neighborhood').apply(wavg, 'percent families below poverty level', 'pop')


# calculate percentage

avgs = pd.DataFrame(employment_by_neighborhood, columns=['unemploymentPct'])
avgs['povertyPct'] = poverty_by_neighborhood
avgs['pop'] = census.groupby('Neighborhood').sum()['pop']
avgs = avgs.reset_index()
avgs['Neighborhood'] = avgs['Neighborhood'].apply(lambda x: x.replace('_', ' '))
avgs = avgs.set_index('Neighborhood').sort_index()

avgs = avgs.join(violent_crimes, how='inner')
avgs['violentCrimePerCapita'] = avgs['Violent Crimes'] / avgs['pop']
avgs['violentCrimesPerThousandPpl'] = avgs['violentCrimePerCapita'] * 1000
avgs['shootings'] = shootings_2015['Shootings2015']


race_totals = census.groupby('Neighborhood').sum()[race_columns].reset_index()
race_totals['Neighborhood'] = race_totals['Neighborhood'].apply(lambda x: x.replace('_', ' '))
race_totals = race_totals.set_index('Neighborhood').sort_index()

for race in race_columns:
    if race == 'pop':
        continue
    col_name = race.lower() + 'Pct'
    avgs[col_name] = (race_totals[race] / race_totals['pop']) * 100

ranked_cols = ['unemploymentPct', 'povertyPct', 'violentCrimePerCapita']

for col in ranked_cols:
    avgs["{}Rank".format(col)] = avgs[col].rank(ascending=False)
    
avgs.to_csv('site/public/csv/averages_2015.csv')

